In [101]:
import pandas as pd
import numpy as np

In [102]:
first= pd.read_csv("korean_air_2020_01.csv")
second = pd.read_csv("korean_air_data_korean_air_2020_02.csv")
third = pd.read_csv("korean_air_data_korean_air_2020_07.csv")
fourth = pd.read_csv("korean_air_data_korean_air_2020_08.csv")
fifth = pd.read_csv("korean_air_data_korean_air_2020_09.csv")
sixth = pd.read_csv("korean_air_data_korean_air_2020_10.csv")

data = pd.concat([first, second, third, fourth, fifth, sixth])

In [103]:
data = data.dropna()
data.columns = ["a","title","text","repoter","spon", "date"]
date = data.date

In [104]:
#q불용어 지정
#기자이름
repoter = data["repoter"].unique

In [105]:
#신문사 이름
spon = data["spon"].unique

In [106]:
#대표적인 조사
stopwords = ['의','가','이','권','은','들','는','좀','잘','걍','과','도','을','것','를','등','으로','자','에','등','와','한','하다''것','로']

In [107]:
#불용어 병합
stopwords.extend(data.repoter.values)
stopwords.extend(data.spon.values)

In [108]:
# 한글 외의 모든 데이터 제거 : 기업명이 영어로 되어있는 경우를 어떻게 처리 해야할까?
data = data.text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer

okt = Okt()
X_train = []
X_train2 =[]

for sen in data :
    temp_X = []
    
    # 토큰화
    temp_X = okt.morphs(sen, stem = True) 
    
    # 불용어 제거
    temp_X = [word for word in temp_X if not word in stopwords] 
    
    X_train.append(temp_X)
    X_train2.extend(temp_X)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
vocab_size = total_cnt - rare_cnt
print('단어 집합의 크기 :',vocab_size)

단어 집합(vocabulary)의 크기 : 18068
등장 빈도가 2번 이하인 희귀 단어의 수: 7557
단어 집합에서 희귀 단어의 비율: 41.82532654416648
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.065955532844695
단어 집합의 크기 : 10511


In [117]:
#from sklearn.feature_extraction.text import CountVectorizer
#vectorizer = CountVectorizer(min_df=1)

In [122]:
#vector_texts = vectorizer.fit_transform(data)
#print(vector_texts.shape) 

(2630, 105730)


In [44]:
#증감 데이터 불러오기
increase = pd.read_csv("대비처리.csv")

In [47]:
increase = increase[["년/월/일","증감"]]

In [48]:
dic_frame = pd.DataFrame(X_train, date)

In [49]:
#날자를 중심으로 데이터 병합
increase = increase.rename(columns ={ "년/월/일" : "date"})

In [50]:
dic_frame2 = pd.merge(dic_frame, increase, on = "date")

In [74]:
#딕셔너리 key 생성
key_list = []
for v in X_train2:
    if v not in key_list:
        key_list.append(v)

,date,0,1,2,3,4,5,6,7,8,...,2757,2758,2759,2760,2761,2762,2763,2764,2765,증감
0,2020/01/31,남매,난,촉발,되다,한진,그룹,경영,권,분쟁,...,None,None,None,None,None,None,None,None,None,0
1,2020/01/31,한진,그룹,경영,권,향배,가르다,월,한진,칼,...,None,None,None,None,None,None,None,None,None,0
2,2020/01/31,줄어들다,비행,수당,처우,대한,정보,아직,없다,신종,...,None,None,None,None,None,None,None,None,None,0
3,2020/01/31,한진,남매,경영,권,분쟁,본격,화,입장,문,...,None,None,None,None,None,None,None,None,None,0
4,2020/01/31,앵커,오늘,우한,교민,이송,전세기,에는,조원태,한진,...,None,None,None,None,None,None,None,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,2020/09/02,투자,귀,재,짐,로저스,회장,단독,화상,인터뷰,...,None,None,None,None,None,None,None,None,None,1
2067,2020/09/02,현대차,월,만대,그치다,소비,악화,전달,감소,신종,...,None,None,None,None,None,None,None,None,None,1
2068,2020/09/02,오다,들다,코로나,경,영,정상화,어려워지다,그룹,사들이다,...,None,None,None,None,None,None,None,None,None,1
2069,2020/09/02,김성미,기자,오다,들다,코로나,경,영,정상화,어려워지다,...,None,None,None,None,None,None,None,None,None,1


In [76]:
#증감을 기준으로 단어 정렬
dic = dic_frame2.drop("date",axis = 1)

In [91]:
dictionary ={}

for k in key_list:
    count = 0
    score = 0
    for x in range(0,len(dic)):
        d = dic.loc[x,:]
        if k in d:
            count = count + 1
            score = score + d[2766]
        
        else :
            pass

        polarity  = score/count
        dictionary[k] = [count,score,polarity]        

ZeroDivisionError: division by zero

In [92]:
first

,Unnamed: 0,0,0.1,0.2,0.3,0.4
0,0,조현아 ‘칼’ 뺐다…조원태 경영권에 ‘적신호’,‘남매의 난’으로 촉발된 한진그룹 경영권 분쟁이 정점으로 치닫고 있다. 그룹 경영권...,김경준,UNK,2020/01/31
1,1,[속보] 조현아-KCGI-반도건설 손잡았다… 절정 치닫는 한진 경영권 분쟁,한진그룹 경영권 향배를 가를 3월 한진칼 주주총회를 앞두고 조현아 전 대한항공 부사...,김경준,UNK,2020/01/31
2,2,우한 교민 전세기 탄 승무원들 비행기서 내리면? 2주간 자가격리,“줄어든 비행수당 등 처우에 대한 정보는 아직 없어”\n\n\n신종 코로나바이러스 ...,박민정,UNK,2020/01/31
3,3,"조현아, KCGI·반도와 ‘반조원태’ 전선",ㆍ한진 남매 경영권 분쟁 본격화\n\n\n\n입장문에 “경영 일선 안 나서”\n조현...,김지원 기자 deepdeep@kyunghyang.com,경향신문,2020/01/31
4,4,'우한' 다녀오니 '우환'…누나 조현아의 '선전포고',◀ 앵커 ▶\n\n오늘 우한 교민을 이송한 전세기에는 조원태 한진그룹 회장도 타고 ...,이준희,MBC,2020/01/31
...,...,...,...,...,...,...
647,647,"[속보]지난해 12월 수출 -5.2%, 13개월 연속 감소",NaN,NaN,머니투데이,NaN
648,648,[씨줄날줄] 마일리지/장세훈 논설위원,국내 항공사 마일리지 약 4900억원어치가 새해 첫날 사라졌다. 대한항공과 아시아나...,장세훈,NaN,2020/01/01
649,649,LCC 신규 취항으로 업계경쟁 격화될듯,◆ 2020 경제기상도 / 업종별 전망 (항공) ◆\n국내 항공업은 올해도 침체기를...,송광섭,매일경제,2020/01/01
650,650,[인투더퓨처-클라우드] 2020년 '클라우드 온리' 시대 열린다,삼성·LG·SK·현기차 등 국내 주요 대기업 3년 내 클라우드 도입 마무리 속도 얻...,강일용,아주경제,2020/01/01


In [123]:
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

tfidf = TfidfTransformer()
np.set_printoptions(precision=2)   
print(tfidf.fit_transform(vector_texts).toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(data)
print(tfidfv.transform(data).toarray())
print(tfidfv.vocabulary_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{'남매의': 16431, '난으로': 16304, '촉발된': 88531, '한진그룹': 98686, '경영권': 5465, '분쟁이': 40189, '정점으로': 78024, '치닫고': 90534, '있다': 71254, '그룹': 11489, '경영권의': 5481, '향배를': 101355, '가늠할': 146, '한진칼': 98764, '주주총회에': 81932, '앞서': 56164, '누나인': 18729, '조현아': 80359, '대한항공': 21815, '부사장이': 38897, '본격적인': 38158, '결집에': 5120, '나서면서다': 15700, '부사장은': 38893, '특히': 94178, '우호세력': 63367, '확보': 103493, '과정에서': 8756, '경영': 5433, '일선에': 70199, '나서지': 15710, '않겠다는': 55522, '조건까지': 79539, '제시하고': 78686, '배수의': 35832, '진까지': 85542, '것으로': 4128, '확인됐다': 103717, '부사장의': 38895, '파상': 94456, '공세가': 8044, '이어지면서': 68100, '동생인': 23461, '조원태': 80020, '회장의': 104797, '경영권도': 5468, '장담할': 73733, '없게': 57803, '됐다법무법인': 23736, '태평양은': 92637, '부사장과': 38858, '강성부펀드로': 1731, '알려진': 55930, '반도건설': 33441, '명의의': 30338, '공동': 7829, '입장문을': 70988, '통해': 93253, '대한항공을': 21911, '비